In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics
from sklearn.svm import SVC

iris = datasets.load_iris()

X_iris = iris.data

y_iris = iris.target

In [250]:
# Separar os dados para implementação do svm
y = list(map(lambda x: -1 if x == 0 else (1 if x == 1 else None) , y_iris))
y = list(filter(lambda x: x != None, y))

X = X_iris[:len(y)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [9]:
#Definição de funções necessárias

# Definir a função do kernel -> RBF
def K(x, u, gamma):
    k = np.exp(-gamma*((np.linalg.norm(x-u))**2))
    return k

#Define a função de descisão do modelo
def decision(a, y, x, u, sigma, b):
    sum = 0
    for i in range(len(a)):
        sum = sum + a[i]*y[i]*K(x[i], u, sigma)
    f = sum + b

    if f >= 0:
        return 1
    else:
        return -1

#Define a optimização para encontrar os suporte vector
def optimizacao_alfa(C):
    a = np.zeros((1, len(y_train)))
    a = a.ravel()
    firstTarget = np.random.randint(len(y_train))

    a[firstTarget] = C

    if y_train[firstTarget] == 1:
        secondTarget = np.random.randint(len(y_train))
        while y_train[secondTarget] != -1:
            secondTarget = np.random.randint(len(y_train))
        a[secondTarget] = C
    elif y_train[firstTarget] == -1:
        secondTarget = np.random.randint(len(y_train))
        while y_train[secondTarget] != 1:
            secondTarget = np.random.randint(len(y_train))
        a[secondTarget] = C
    return a


In [10]:
#Teste simples de funcionamento para um classificar com 2 classes
Y_pred = []
a = optimizacao_alfa(10)

for i in range(len(X_test)):
    y_pred = decision(a, y_train, X_train, X_test[i], 1, 1)
    Y_pred.append(y_pred)

print(f'Teste: {y_test}')
print(f'Previsão: {Y_pred}')

accuracy = metrics.accuracy_score(y_test, Y_pred)

print(f'Accuracy: {accuracy}')


Teste: [-1, -1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, -1, -1]
Previsão: [-1, -1, -1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, -1, -1]
Accuracy: 1.0


In [27]:
#One-vs-one classifier method

X_train, X_test, y_train, y_test = train_test_split(X_iris, y_iris, test_size=0.3)

#Pega os valores das classes no dataset em estudo
y_set = list(set(y_train))

#Define os duelos entre as classes sem permetir repetição
setClasses = []
for i in range(len(y_set)-1):
    for j in range(i+1,len(y_set)):
        setClasses.append([y_set[i], y_set[j]])

#Para cada duelo, iremos mapear os valores da classe como -1 e 1

for s in setClasses:
    x = X_train.copy().tolist()
    y = list(map(lambda x: -1 if x == s[0] else (1 if x == s[1] else None) , y_train))
    for i in range(len(y)):
        if y[i] == None:
            x[i] = None
    y = list(filter(lambda x: x != None, y))
    x = list(filter(lambda i: i != None, x))

    #Com isso podemos rodar encontrar a classificação para cada classe:

    Y_pred = []
    a = optimizacao_alfa(10)

    for i in range(len(X_test)):
        y_pred = decision(a, y, x, X_test[i], 1, 1)
        Y_pred.append(y_pred)

    print(f'Classe = {str(s)}:')
    print(f'\tTeste: {y_test}')
    print(f'\tPrevisão: {Y_pred}')

    accuracy = metrics.accuracy_score(y_test, Y_pred)

    print(f'\tAccuracy {str(s)}: {accuracy}')


[-1, 1, -1, None, -1, 1, 1, None, -1, -1, None, None, -1, None, -1, None, -1, None, 1, None, -1, 1, None, 1, None, -1, None, 1, 1, -1, None, None, -1, -1, 1, -1, 1, -1, None, 1, None, -1, 1, -1, -1, None, None, 1, -1, 1, None, -1, 1, None, None, None, None, None, None, 1, 1, 1, None, -1, -1, 1, -1, 1, -1, -1, -1, 1, None, -1, -1, -1, -1, 1, None, -1, None, -1, None, -1, None, 1, -1, 1, None, -1, -1, 1, -1, 1, 1, None, 1, -1, None, None, None, None, None, 1, 1]
[[5.5, 4.2, 1.4, 0.2], [6.7, 3.1, 4.4, 1.4], [5.5, 3.5, 1.3, 0.2], None, [5.4, 3.7, 1.5, 0.2], [6.7, 3.0, 5.0, 1.7], [5.7, 2.9, 4.2, 1.3], None, [5.1, 3.8, 1.5, 0.3], [5.1, 3.5, 1.4, 0.2], None, None, [5.4, 3.9, 1.7, 0.4], None, [5.1, 3.3, 1.7, 0.5], None, [5.0, 3.4, 1.6, 0.4], None, [5.2, 2.7, 3.9, 1.4], None, [4.9, 3.6, 1.4, 0.1], [6.1, 3.0, 4.6, 1.4], None, [6.5, 2.8, 4.6, 1.5], None, [4.8, 3.4, 1.6, 0.2], None, [6.4, 3.2, 4.5, 1.5], [6.6, 3.0, 4.4, 1.4], [4.7, 3.2, 1.3, 0.2], None, None, [5.1, 3.7, 1.5, 0.4], [5.8, 4.0, 1.2, 

In [ ]:
#Encontrado as classificações para cada duelo de classes, 